## Search engine with tools and agents

In [1]:
## arxiv research pprr
## tools creation

from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [3]:
## used the inbuilt tools of wikipedia
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [4]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

'arxiv'

In [5]:
tools=[wiki,arxiv]

In [18]:
## custom tools[RAG tools]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
import openai

In [12]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs= loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100).split_documents(docs)
vector_store = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever=vector_store.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x75e3f85782b0>, search_kwargs={})

In [13]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, "langsmith-search", "Search any information about langsmith")

retriever_tool.name

'langsmith-search'

In [15]:
tools=[wiki, arxiv, retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/ranker/Desktop/Gen-AI/venv/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Search any information about langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x75e3fb57d240>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x75e3f85782b0>, search

In [22]:
## run all these tools with agents and LLM models
## tools, LLM --> AgentsExecutor

load_dotenv()

groq_api=os.getenv("GROQ_API_KEY")
openai.api_key=os.getenv("OPENAI_API_KEY")

llm = ChatGroq(groq_api_key=groq_api, model_name="Llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x75e3d4081ba0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x75e3d4082860>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [24]:
## prompt template

from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [25]:
## Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm, tools, prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [27]:
## Agent Executor
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [28]:
agent_executor.invoke({"input": "Tell me about the langsmith"})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `langsmith-search` with `{'query': 'langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith​PythonTypeScriptpip install -U langsmithyarn add langsmith2. Create an API key​To create an API key head to the Settings page. Then click Create API Key.3. Set up your environment​Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-openai-api-key>4. Log your first trace​Tracing to LangSmith for LangChain u

{'input': 'Tell me about the langsmith',
 'output': "It looks like the Langsmith platform is a tool for building production-grade LLM (Large Language Model) applications. It allows users to closely monitor and evaluate their applications, making it easier to ship them quickly and with confidence. Langsmith seems to be a self-contained platform that doesn't require the use of LangChain. \n\nIt appears to have several steps to get started, including installing Langsmith, creating an API key, setting up the environment, and logging the first trace. Langsmith also provides a community and documentation for users to learn more and get started with the platform."}

In [29]:
agent_executor.invoke({"input": "Tell me what is machine learning"})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `wikipedia` with `{'query': 'Machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks witThank you for providing the result of the tool call. Here's a response based on the provided information:

Machine learning (ML) is a subfield of artificial intelligence that involves the development of algorithms and statistical models that enable machines to learn from data and make predictions or decisions without being explicitly programmed.

Machine learning involves training models on large datasets, which allows the models to learn patterns and relationships in the data. Once trained, these models can be used to make predictions or classify new, unseen data. The key characteristics of machine learning are its ability to learn from data, improve over time, and adapt to new 

{'input': 'Tell me what is machine learning',
 'output': "Thank you for providing the result of the tool call. Here's a response based on the provided information:\n\nMachine learning (ML) is a subfield of artificial intelligence that involves the development of algorithms and statistical models that enable machines to learn from data and make predictions or decisions without being explicitly programmed.\n\nMachine learning involves training models on large datasets, which allows the models to learn patterns and relationships in the data. Once trained, these models can be used to make predictions or classify new, unseen data. The key characteristics of machine learning are its ability to learn from data, improve over time, and adapt to new data.\n\nMachine learning has many applications, including image recognition, natural language processing, speech recognition, and recommender systems. It is also used in areas such as healthcare, finance, and education to analyze data and make predi

In [30]:
agent_executor.invoke({"input": "What's the paper 2409.17426 about?"})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `arxiv` with `{'query': '2409.17426'}`


Published: 2024-09-25
Title: Exploring the Use of ChatGPT for a Systematic Literature Review: a Design-Based Research
Authors: Qian Huang, Qiyun Wang
Summary: ChatGPT has been used in several educational contexts,including learning,
teaching and rese
Invoking: `wikipedia` with `{'query': 'ChatGPT'}`


Page: ChatGPT
Summary: ChatGPT is a generative artificial intelligence chatbot developed by OpenAI. Launched in 2022 based on the GPT-3.5 large language model (LLM), it was later updated to use the GPT-4 architecture. ChatGPT can generate human-like The paper "Exploring the Use of ChatGPT for a Systematic Literature Review: a Design-Based Research" by Qian Huang and Qiyun Wang appears to be a study evaluating the use of ChatGPT in educational contexts, specifically for systematic literature reviews. The paper was published in 2024.

> Finished chain.


{'input': "What's the paper 2409.17426 about?",
 'output': 'The paper "Exploring the Use of ChatGPT for a Systematic Literature Review: a Design-Based Research" by Qian Huang and Qiyun Wang appears to be a study evaluating the use of ChatGPT in educational contexts, specifically for systematic literature reviews. The paper was published in 2024.'}